In [ ]:
import shapely
print(f'Using shapely {shapely.__version__}')

In [ ]:
import math
import os
import random
import itertools
from decimal import Decimal, getcontext

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.patches import Rectangle
from shapely import affinity, touches
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from tqdm import tqdm

import sys
sys.path.append('/kaggle/usr/lib/santa-2025-metric')
from metric import score as competition_score

pd.set_option('display.float_format', '{:.12f}'.format)

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

In [ ]:
# Build the index of the submission, in the format:
#  <trees_in_problem>_<tree_index>

index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]

In [ ]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                # Start at Tip
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                # Right side - Top Tier
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                # Right side - Middle Tier
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                # Right side - Bottom Tier
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                # Right Trunk
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                # Left Trunk
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Bottom Tier
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Middle Tier
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                # Left side - Top Tier
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

In [ ]:
def plot_results(placed_trees, num_trees):
    """Plots the arrangement of trees and the bounding square."""
    _, ax = plt.subplots(figsize=(6, 6))
    colors = plt.cm.viridis([i / num_trees for i in range(num_trees)])

    all_polygons = [t.polygon for t in placed_trees]
    bounds = unary_union(all_polygons).bounds

    for i, tree in enumerate(placed_trees):
        # Rescale for plotting
        x_scaled, y_scaled = tree.polygon.exterior.xy
        x = [Decimal(val) / scale_factor for val in x_scaled]
        y = [Decimal(val) / scale_factor for val in y_scaled]
        ax.plot(x, y, color=colors[i])
        ax.fill(x, y, alpha=0.5, color=colors[i])

    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor

    width = maxx - minx
    height = maxy - miny

    side_length = max(width, height)

    square_x = minx if width >= height else minx - (side_length - width) / 2
    square_y = miny if height >= width else miny - (side_length - height) / 2
    bounding_square = Rectangle(
        (float(square_x), float(square_y)),
        float(side_length),
        float(side_length),
        fill=False,
        edgecolor='red',
        linewidth=2,
        linestyle='--',
    )
    ax.add_patch(bounding_square)

    padding = 0.5
    ax.set_xlim(
        float(square_x - Decimal(str(padding))),
        float(square_x + side_length + Decimal(str(padding))))
    ax.set_ylim(float(square_y - Decimal(str(padding))),
                float(square_y + side_length + Decimal(str(padding))))
    ax.set_aspect('equal', adjustable='box')
    ax.axis('off')
    plt.title(f'{num_trees} Trees: {side_length:.12f}')
    plt.show()
    plt.close()

In [ ]:
def plot_results(placed_trees, num_trees):
    """Plots the arrangement of trees with tree numbers displayed."""
    fig, ax = plt.subplots(figsize=(8, 8))
    colors = plt.cm.viridis([i / num_trees for i in range(num_trees)])

    all_polygons = [t.polygon for t in placed_trees]
    bounds = unary_union(all_polygons).bounds

    for i, tree in enumerate(placed_trees):
        # Rescale for plotting
        x_scaled, y_scaled = tree.polygon.exterior.xy
        x = [Decimal(val) / scale_factor for val in x_scaled]
        y = [Decimal(val) / scale_factor for val in y_scaled]
        
        # Plot the tree
        ax.plot(x, y, color=colors[i], linewidth=1)
        ax.fill(x, y, alpha=0.5, color=colors[i])
        
        # Get center coordinates
        center_x = float(tree.center_x)
        center_y = float(tree.center_y)
        
        # Add tree number (just the index)
        ax.text(center_x, center_y, str(i), 
                fontsize=10, fontweight='bold', 
                ha='center', va='center',
                bbox=dict(boxstyle="circle,pad=0.3", 
                         facecolor='white', 
                         edgecolor='black', 
                         alpha=0.8))

    # Rest of the plotting code remains the same
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor

    width = maxx - minx
    height = maxy - miny
    side_length = max(width, height)

    square_x = minx if width >= height else minx - (side_length - width) / 2
    square_y = miny if height >= width else miny - (side_length - height) / 2
    
    bounding_square = Rectangle(
        (float(square_x), float(square_y)),
        float(side_length),
        float(side_length),
        fill=False,
        edgecolor='red',
        linewidth=2,
        linestyle='--',
    )
    ax.add_patch(bounding_square)

    padding = 0.5
    ax.set_xlim(
        float(square_x - Decimal(str(padding))),
        float(square_x + side_length + Decimal(str(padding))))
    ax.set_ylim(float(square_y - Decimal(str(padding))),
                float(square_y + side_length + Decimal(str(padding))))
    ax.set_aspect('equal', adjustable='box')
    ax.axis('off')
    plt.title(f'{num_trees} Trees - Side Length: {side_length:.12f}')
    plt.show()
    plt.close()

In [ ]:
def get_BB(placed_trees, num_trees):
    all_polygons = [t.polygon for t in placed_trees]
    bounds = unary_union(all_polygons).bounds

    for i, tree in enumerate(placed_trees):
        # Rescale for plotting
        x_scaled, y_scaled = tree.polygon.exterior.xy
        x = [Decimal(val) / scale_factor for val in x_scaled]
        y = [Decimal(val) / scale_factor for val in y_scaled]

        # Get center coordinates
        center_x = float(tree.center_x)
        center_y = float(tree.center_y)

    # Rest of the plotting code remains the same
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor

    width = maxx - minx
    height = maxy - miny
    side_length = float(max(width, height))
    return float(minx),float(miny),float(maxx),float(maxy),side_length

In [ ]:
# Let's take highest public score as baseline
# https://www.kaggle.com/code/jiweiliu/super-fast-simulated-annealing-with-translations
# Thanks to Jiwei for share
submission_df = pd.read_csv('/kaggle/input/super-fast-simulated-annealing-with-translations/submission.csv')[['id','x','y','deg']]
submission_df['group'] = submission_df['id'].apply(lambda v:int(v.split('_')[0]))
submission = [submission_df[submission_df.group == k] for k in range(1,201)]

In [ ]:
# From https://www.kaggle.com/competitions/santa-2025/discussion/651069
# Thanks to Paul and Stanislav for share
Paul_Jurczak_scores = '''1  0.8132 x 0.8132  0.0000%  score: 0.6612
  2  0.9495 x 0.9495  0.0000%  score: 0.4508
  3  1.1420 x 1.1420  0.0000%  score: 0.4347
  4  1.2909 x 1.2909  0.0000%  score: 0.4166
  5  1.4440 x 1.4440  0.0000%  score: 0.4171
  6  1.5492 x 1.5492  0.0001%  score: 0.4000
  7  1.6738 x 1.6738  0.0000%  score: 0.4002
  8  1.7593 x 1.7500  0.5332%  score: 0.3869
  9  1.8620 x 1.8521  0.5355%  score: 0.3852
 10  1.9420 x 1.9407  0.0710%  score: 0.3772
 11  2.0350 x 2.0341  0.0423%  score: 0.3765
 12  2.1169 x 2.1157  0.0558%  score: 0.3734
 13  2.2030 x 2.2030  0.0000%  score: 0.3733
 14  2.3064 x 2.2819  1.0737%  score: 0.3800
 15  2.3741 x 2.3633  0.4589%  score: 0.3758
 16  2.4431 x 2.4408  0.0967%  score: 0.3731
 17  2.5109 x 2.5109  0.0017%  score: 0.3709
 18  2.5800 x 2.5717  0.3216%  score: 0.3698
 19  2.6400 x 2.6245  0.5889%  score: 0.3668
 20  2.7300 x 2.7250  0.1807%  score: 0.3726
 21  2.8100 x 2.7907  0.6908%  score: 0.3760
 22  2.8917 x 2.8841  0.2644%  score: 0.3801
 23  2.9182 x 2.9182  0.0000%  score: 0.3703
 24  2.9684 x 2.9683  0.0025%  score: 0.3671
 25  3.0451 x 3.0450  0.0037%  score: 0.3709
 26  3.1008 x 3.0987  0.0662%  score: 0.3698
 27  3.1616 x 3.1605  0.0330%  score: 0.3702
 28  3.2061 x 3.1931  0.4084%  score: 0.3671
 29  3.2774 x 3.2643  0.3987%  score: 0.3704
 30  3.2900 x 3.2882  0.0554%  score: 0.3608
 31  3.3700 x 3.3700  0.0000%  score: 0.3663
 32  3.4294 x 3.4294  0.0000%  score: 0.3675
 33  3.4843 x 3.4843  0.0000%  score: 0.3679
 34  3.5455 x 3.5455  0.0001%  score: 0.3697
 35  3.5826 x 3.5826  0.0000%  score: 0.3667
 36  3.6016 x 3.6016  0.0000%  score: 0.3603
 37  3.6733 x 3.6733  0.0000%  score: 0.3647
 38  3.7274 x 3.7274  0.0000%  score: 0.3656
 39  3.7649 x 3.7649  0.0000%  score: 0.3634
 40  3.8399 x 3.8399  0.0000%  score: 0.3686
 41  3.8700 x 3.8635  0.1679%  score: 0.3653
 42  3.9301 x 3.9177  0.3153%  score: 0.3678
 43  3.9758 x 3.9756  0.0070%  score: 0.3676
 44  4.0388 x 4.0020  0.9174%  score: 0.3707
 45  4.0614 x 4.0511  0.2535%  score: 0.3666
 46  4.1044 x 4.0965  0.1940%  score: 0.3662
 47  4.1717 x 4.0684  2.5405%  score: 0.3703
 48  4.2123 x 4.1918  0.4893%  score: 0.3697
 49  4.2377 x 4.1958  0.9992%  score: 0.3665
 50  4.2500 x 4.2474  0.0617%  score: 0.3612
 51  4.3068 x 4.3050  0.0430%  score: 0.3637
 52  4.3469 x 4.3396  0.1679%  score: 0.3634
 53  4.3900 x 4.3872  0.0646%  score: 0.3636
 54  4.4500 x 4.4432  0.1514%  score: 0.3667
 55  4.4661 x 4.4265  0.8950%  score: 0.3627
 56  4.4850 x 4.4850  0.0000%  score: 0.3592
 57  4.5776 x 4.5182  1.3154%  score: 0.3676
 58  4.6250 x 4.5144  2.4495%  score: 0.3688
 59  4.6508 x 4.5963  1.1856%  score: 0.3666
 60  4.6951 x 4.6840  0.2375%  score: 0.3674
 61  4.7324 x 4.6996  0.6977%  score: 0.3671
 62  4.7371 x 4.6856  1.0997%  score: 0.3619
 63  4.7443 x 4.7422  0.0453%  score: 0.3573
 64  4.7456 x 4.7456  0.0000%  score: 0.3519
 65  4.8774 x 4.8769  0.0110%  score: 0.3660
 66  4.8780 x 4.8778  0.0036%  score: 0.3605
 67  4.8826 x 4.8811  0.0293%  score: 0.3558
 68  4.9486 x 4.9486  0.0001%  score: 0.3601
 69  4.9514 x 4.9514  0.0002%  score: 0.3553
 70  5.0225 x 4.9062  2.3701%  score: 0.3604
 71  5.0240 x 4.8793  2.9659%  score: 0.3555
 72  5.0244 x 4.9107  2.3158%  score: 0.3506
 73  5.1584 x 5.1463  0.2361%  score: 0.3645
 74  5.1768 x 5.1574  0.3767%  score: 0.3622
 75  5.2345 x 5.2214  0.2501%  score: 0.3653
 76  5.2556 x 5.2108  0.8612%  score: 0.3634
 77  5.3065 x 5.2774  0.5515%  score: 0.3657
 78  5.3076 x 5.2977  0.1865%  score: 0.3612
 79  5.3489 x 5.3230  0.4869%  score: 0.3622
 80  5.4090 x 5.3923  0.3107%  score: 0.3657
 81  5.4390 x 5.4163  0.4194%  score: 0.3652
 82  5.4626 x 5.4626  0.0000%  score: 0.3639
 83  5.5018 x 5.4618  0.7316%  score: 0.3647
 84  5.5408 x 5.4685  1.3226%  score: 0.3655
 85  5.5655 x 5.5654  0.0017%  score: 0.3644
 86  5.5656 x 5.5656  0.0010%  score: 0.3602
 87  5.5754 x 5.5753  0.0027%  score: 0.3573
 88  5.5755 x 5.5753  0.0040%  score: 0.3533
 89  5.5772 x 5.5772  0.0000%  score: 0.3495
 90  5.5775 x 5.5775  0.0001%  score: 0.3457
 91  5.7532 x 5.7376  0.2731%  score: 0.3637
 92  5.8181 x 5.8181  0.0000%  score: 0.3679
 93  5.8384 x 5.8384  0.0001%  score: 0.3665
 94  5.8398 x 5.7782  1.0663%  score: 0.3628
 95  5.8670 x 5.8584  0.1469%  score: 0.3623
 96  5.8823 x 5.8823  0.0000%  score: 0.3604
 97  5.8862 x 5.8862  0.0000%  score: 0.3572
 98  5.8878 x 5.8878  0.0000%  score: 0.3537
 99  5.8884 x 5.8884  0.0000%  score: 0.3502
100  5.8896 x 5.8895  0.0015%  score: 0.3469
101  5.9696 x 5.9696  0.0000%  score: 0.3528
102  5.9698 x 5.9698  0.0011%  score: 0.3494
103  5.9707 x 5.9707  0.0000%  score: 0.3461
104  5.9762 x 5.9762  0.0000%  score: 0.3434
105  6.0264 x 6.0263  0.0019%  score: 0.3459
106  6.0918 x 6.0918  0.0000%  score: 0.3501
107  6.1013 x 6.1013  0.0000%  score: 0.3479
108  6.1016 x 6.0346  1.1096%  score: 0.3447
109  6.1016 x 6.0586  0.7109%  score: 0.3416
110  6.1030 x 6.1030  0.0000%  score: 0.3386
111  6.3646 x 6.3646  0.0000%  score: 0.3649
112  6.3648 x 6.3648  0.0012%  score: 0.3617
113  6.4043 x 6.4043  0.0000%  score: 0.3630
114  6.4299 x 6.4299  0.0000%  score: 0.3627
115  6.4499 x 6.4499  0.0000%  score: 0.3617
116  6.4947 x 6.4947  0.0000%  score: 0.3636
117  6.5250 x 6.5250  0.0000%  score: 0.3639
118  6.5531 x 6.5531  0.0000%  score: 0.3639
119  6.5552 x 6.5552  0.0000%  score: 0.3611
120  6.5974 x 6.5972  0.0032%  score: 0.3627
121  6.6519 x 6.6519  0.0000%  score: 0.3657
122  6.6519 x 6.6519  0.0000%  score: 0.3627
123  6.6683 x 6.6682  0.0007%  score: 0.3615
124  6.7159 x 6.7159  0.0000%  score: 0.3637
125  6.7215 x 6.7215  0.0000%  score: 0.3614
126  6.7264 x 6.7264  0.0000%  score: 0.3591
127  6.7288 x 6.7288  0.0000%  score: 0.3565
128  6.7305 x 6.7305  0.0000%  score: 0.3539
129  6.7312 x 6.7312  0.0001%  score: 0.3512
130  6.7321 x 6.7321  0.0000%  score: 0.3486
131  6.7325 x 6.7325  0.0000%  score: 0.3460
132  6.7327 x 6.7327  0.0003%  score: 0.3434
133  6.9877 x 6.9876  0.0003%  score: 0.3671
134  7.0025 x 7.0024  0.0014%  score: 0.3659
135  7.0035 x 7.0034  0.0014%  score: 0.3633
136  7.0370 x 7.0369  0.0019%  score: 0.3641
137  7.0710 x 7.0710  0.0006%  score: 0.3650
138  7.0938 x 7.0933  0.0066%  score: 0.3646
139  7.0938 x 7.0938  0.0002%  score: 0.3620
140  7.0966 x 7.0966  0.0000%  score: 0.3597
141  7.0970 x 7.0970  0.0000%  score: 0.3572
142  7.0986 x 7.0986  0.0000%  score: 0.3549
143  7.0988 x 7.0988  0.0000%  score: 0.3524
144  7.0989 x 7.0989  0.0000%  score: 0.3500
145  7.1124 x 7.1122  0.0027%  score: 0.3489
146  7.1141 x 7.1139  0.0024%  score: 0.3466
147  7.1180 x 7.1172  0.0111%  score: 0.3447
148  7.1185 x 7.1177  0.0117%  score: 0.3424
149  7.1187 x 7.1187  0.0000%  score: 0.3401
150  7.1273 x 7.1273  0.0000%  score: 0.3387
151  7.1806 x 7.1806  0.0000%  score: 0.3415
152  7.1808 x 7.1808  0.0002%  score: 0.3392
153  7.1817 x 7.1815  0.0027%  score: 0.3371
154  7.1846 x 7.1846  0.0001%  score: 0.3352
155  7.1848 x 7.1848  0.0000%  score: 0.3330
156  7.1848 x 7.1848  0.0001%  score: 0.3309
157  7.4088 x 7.4086  0.0031%  score: 0.3496
158  7.5752 x 7.5739  0.0166%  score: 0.3632
159  7.5780 x 7.5780  0.0000%  score: 0.3612
160  7.5806 x 7.5806  0.0000%  score: 0.3592
161  7.5824 x 7.5653  0.2261%  score: 0.3571
162  7.5896 x 7.5888  0.0109%  score: 0.3556
163  7.6572 x 7.6569  0.0028%  score: 0.3597
164  7.7188 x 7.5156  2.7032%  score: 0.3633
165  7.7229 x 7.5220  2.6703%  score: 0.3615
166  7.7238 x 7.5010  2.9692%  score: 0.3594
167  7.7250 x 7.5217  2.7026%  score: 0.3573
168  7.7310 x 7.5132  2.8984%  score: 0.3558
169  7.7345 x 7.7298  0.0614%  score: 0.3540
170  7.7588 x 7.7588  0.0001%  score: 0.3541
171  7.7631 x 7.7628  0.0035%  score: 0.3524
172  7.7934 x 7.7766  0.2158%  score: 0.3531
173  7.7938 x 7.7938  0.0000%  score: 0.3511
174  7.8173 x 7.8168  0.0068%  score: 0.3512
175  7.8568 x 7.8568  0.0002%  score: 0.3527
176  7.8799 x 7.8799  0.0000%  score: 0.3528
177  7.8855 x 7.8852  0.0048%  score: 0.3513
178  7.8870 x 7.8870  0.0000%  score: 0.3495
179  7.8888 x 7.8887  0.0003%  score: 0.3477
180  7.8890 x 7.8890  0.0001%  score: 0.3458
181  7.8893 x 7.8893  0.0000%  score: 0.3439
182  7.8905 x 7.8904  0.0005%  score: 0.3421
183  8.0090 x 8.0090  0.0000%  score: 0.3505
184  8.1123 x 8.1123  0.0000%  score: 0.3577
185  8.1134 x 8.1124  0.0129%  score: 0.3558
186  8.1207 x 8.1183  0.0299%  score: 0.3545
187  8.1290 x 8.1290  0.0002%  score: 0.3534
188  8.1302 x 8.1210  0.1136%  score: 0.3516
189  8.1966 x 8.1966  0.0000%  score: 0.3555
190  8.2093 x 8.2062  0.0367%  score: 0.3547
191  8.2104 x 8.2104  0.0000%  score: 0.3529
192  8.2174 x 8.2173  0.0002%  score: 0.3517
193  8.2194 x 8.2194  0.0008%  score: 0.3500
194  8.2220 x 8.2213  0.0091%  score: 0.3485
195  8.2460 x 8.2460  0.0003%  score: 0.3487
196  8.2518 x 8.2414  0.1262%  score: 0.3474
197  8.2733 x 8.2729  0.0046%  score: 0.3474
198  8.2743 x 8.2743  0.0001%  score: 0.3458
199  8.2778 x 8.2778  0.0003%  score: 0.3443
200  8.2783 x 8.2783  0.0000%  score: 0.3427'''
Stanislav_Chistyakov_scores = '''│  ┆ 0.81317279836452800 ┆ 0.6612499999999973115515687 │
│  ┆ 0.94950427779530625 ┆ 0.4507791867757930507160872 │
│  ┆ 1.14203131659746350 ┆ 0.4347451760297786368969650 │
│  ┆ 1.29094086608578325 ┆ 0.4166320799325780405234870 │
│  ┆ 1.44404852669113150 ┆ 0.4170552294877655048344002 │
│  ┆ 1.54919474855695500 ┆ 0.4000007281594078376758132 │
│  ┆ 1.67378022872329025 ┆ 0.4002200362949985462333576 │
│  ┆ 1.76716255039585775 ┆ 0.3903579349401990602514035 │
│  ┆ 1.86756712498506850 ┆ 0.3875341073694438297718164 │
│ ┆ 1.94248561650837975 ┆ 0.3773250370341940159939802 │
│ ┆ 2.03636944338114650 ┆ 0.3769818645396764019052793 │
│ ┆ 2.12204142398642925 ┆ 0.3752549837595293657242240 │
│ ┆ 2.20301640059928475 ┆ 0.3733293277930329434922263 │
│ ┆ 2.31344818640441350 ┆ 0.3822887507984192822929551 │
│ ┆ 2.38702885303260500 ┆ 0.3798604496806769173670129 │
│ ┆ 2.45502029023923650 ┆ 0.3766952890928965639546502 │
│ ┆ 2.51090951588057650 ┆ 0.3708627409964488853438947 │
│ ┆ 2.58406775609795150 ┆ 0.3709670093391723423118479 │
│ ┆ 2.68411017520855200 ┆ 0.3791814438241096692356155 │
│ ┆ 2.75197146028278800 ┆ 0.3786673459105490305270464 │
│ ┆ 2.81464150434110500 ┆ 0.3772479427599789807885806 │
│ ┆ 2.89257042078979900 ┆ 0.3803165290558215841324974 │
│ ┆ 2.91823424935247650 ┆ 0.3702648319171222604199799 │
│ ┆ 2.96823955801143000 ┆ 0.3671019197393287216790005 │
│ ┆ 3.05376532021011050 ┆ 0.3730193052367183486631946 │
│ ┆ 3.12686078006501900 ┆ 0.3760483976118776585428410 │
│ ┆ 3.19533446589428100 ┆ 0.3781541610715514824884841 │
│ ┆ 3.23729149483722800 ┆ 0.3742877222337662215206504 │
│ ┆ 3.30675808727982950 ┆ 0.3770568637169157411479531 │
│ ┆ 3.35533353751444200 ┆ 0.3752754382656393120049120 │
│ ┆ 3.40086693504983700 ┆ 0.3730934164488797495445700 │
│ ┆ 3.42936908041894800 ┆ 0.3675178840541719073384750 │
│ ┆ 3.50638721431635800 ┆ 0.3725682211127645178361609 │
│ ┆ 3.55563250993975300 ┆ 0.3718388984041314034218732 │
│ ┆ 3.59688072821090500 ┆ 0.3696443135135717212549189 │
│ ┆ 3.60159929807504500 ┆ 0.3603199306637404678511792 │
│ ┆ 3.70472466417591900 ┆ 0.3709455361446858327665778 │
│ ┆ 3.75606347871153350 ┆ 0.3712634962134364860514029 │
│ ┆ 3.76467272261258550 ┆ 0.3634041207277758237913005 │
│ ┆ 3.86314178851262000 ┆ 0.3730966119538121107747975 │
│ ┆ 3.90003161616342250 ┆ 0.3709816245627872509648349 │
│ ┆ 3.93486278513832600 ┆ 0.3686463128063462829574988 │
│ ┆ 4.01178608922249750 ┆ 0.3742890145506776900400944 │
│ ┆ 4.03634553192427000 ┆ 0.3702746648428413211302050 │
│ ┆ 4.06242812318821250 ┆ 0.3667404945793445030052378 │
│ ┆ 4.11509259793751000 ┆ 0.3681301541217409854588230 │
│ ┆ 4.18961430986824800 ┆ 0.3734652779883573610824494 │
│ ┆ 4.20179766722424900 ┆ 0.3678146590893987653980094 │
│ ┆ 4.25789392069485300 ┆ 0.3699930742834732067034763 │
│ ┆ 4.30876799970273400 ┆ 0.3713096335052459908702474 │
│ ┆ 4.34205097943222400 ┆ 0.3696746413331046220761061 │
│ ┆ 4.37468493179811050 ┆ 0.3680359279327199753246679 │
│ ┆ 4.39129902892780100 ┆ 0.3638397577634386423361447 │
│ ┆ 4.48097708373984000 ┆ 0.3718362152778074270318667 │
│ ┆ 4.48428156977260600 ┆ 0.3656142035818594082435373 │
│ ┆ 4.48503556472240600 ┆ 0.3592061431575862732109971 │
│ ┆ 4.58171970682640200 ┆ 0.3682834293319545805901502 │
│ ┆ 4.62623145307603800 ┆ 0.3690003009901729307491760 │
│ ┆ 4.65680326827641300 ┆ 0.3675562149055912160595215 │
│ ┆ 4.69903069564837600 ┆ 0.3680148246440943412870505 │
│ ┆ 4.74098754199833200 ┆ 0.3684748012030063250747523 │
│ ┆ 4.74321801824242800 ┆ 0.3628728575577359038611965 │
│ ┆ 4.74432992005813000 ┆ 0.3572804188945838443889298 │
│ ┆ 4.74533844575450900 ┆ 0.3518474525743096742994733 │
│ ┆ 4.80825441619282900 ┆ 0.3556817004744283513477371 │
│ ┆ 4.87800241379339000 ┆ 0.3605289022571839278565092 │
│ ┆ 4.87912512465745400 ┆ 0.3553113728666211348504603 │
│ ┆ 4.94862118626556400 ┆ 0.3601301712522999686668872 │
│ ┆ 4.95126284178417900 ┆ 0.3552899091077209270782610 │
│ ┆ 5.01356140436901500 ┆ 0.3590828279339801419875756 │
│ ┆ 5.01397114386911000 ┆ 0.3540831919936917094357861 │
│ ┆ 5.01479033498624500 ┆ 0.3492794736648813242067269 │
│ ┆ 5.20214572258926300 ┆ 0.3707167139596406201851096 │
│ ┆ 5.22127864973115800 ┆ 0.3684020370018706014922389 │
│ ┆ 5.23723278633466600 ┆ 0.3657147634434502572164127 │
│ ┆ 5.30529198490869300 ┆ 0.3703437242781107846639414 │
│ ┆ 5.31289990085260900 ┆ 0.3665831864477878252260408 │
│ ┆ 5.33861073704405100 ┆ 0.3653944179702823775917065 │
│ ┆ 5.39186048238512800 ┆ 0.3680020185000808254629332 │
│ ┆ 5.42639556719980000 ┆ 0.3680721106465704894701640 │
│ ┆ 5.44613903878553700 ┆ 0.3661781534541216405877022 │
│ ┆ 5.45281690922793200 ┆ 0.3626001493361226488836138 │
│ ┆ 5.53145210523741000 ┆ 0.3686381011148840375126802 │
│ ┆ 5.54082397980458200 ┆ 0.3654848854187796061784632 │
│ ┆ 5.56518769999666200 ┆ 0.3643684016022839624342919 │
│ ┆ 5.56561141543772800 ┆ 0.3601864003215203503735099 │
│ ┆ 5.57382333334604400 ┆ 0.3570977764523310934713684 │
│ ┆ 5.57452201268097800 ┆ 0.3531283598848270664299909 │
│ ┆ 5.57456488567400000 ┆ 0.3491659962313435583194863 │
│ ┆ 5.57457354322491600 ┆ 0.3452874465424799379449484 │
│ ┆ 5.78014992872949500 ┆ 0.3671443208636438036042295 │
│ ┆ 5.79372113962471300 ┆ 0.3648609200405900342305052 │
│ ┆ 5.82767197112080800 ┆ 0.3651802215374955229242982 │
│ ┆ 5.83571023302797500 ┆ 0.3622927013177385348567780 │
│ ┆ 5.87856365229103200 ┆ 0.3637632696214450250276045 │
│ ┆ 5.88218996647257400 ┆ 0.3604183208507356378980030 │
│ ┆ 5.88341281254699400 ┆ 0.3568509930189910346358089 │
│ ┆ 5.88430146886215800 ┆ 0.3533163650658505121922367 │
│ ┆ 5.88439361156592100 ┆ 0.3497584664226042742907245 │
│┆ 5.88783054906076500 ┆ 0.3466654857445318944762366 │
│┆ 5.96630916278594500 ┆ 0.3524440101578566728046920 │
│┆ 5.96812439559848800 ┆ 0.3492010666797825263841932 │
│┆ 5.96814167461807200 ┆ 0.3458127674593495611441463 │
│┆ 5.96878404803744200 ┆ 0.3425613751164060839293445 │
│┆ 6.02636741433229400 ┆ 0.3458771829764390367909129 │
│┆ 6.05827579317605300 ┆ 0.3462519394922918311830288 │
│┆ 6.09759333401812400 ┆ 0.3474826585706753375623473 │
│┆ 6.09761942349369100 ┆ 0.3442681725348845610218919 │
│┆ 6.09765503577995800 ┆ 0.3411137333520429438273889 │
│┆ 6.09765503587634100 ┆ 0.3380126994231645586165622 │
│┆ 6.39069922347803300 ┆ 0.3679372663510156213744615 │
│┆ 6.39840646697525100 ┆ 0.3655321903270599443365808 │
│┆ 6.43690800347696400 ┆ 0.3666706605772194228912680 │
│┆ 6.45924939398242900 ┆ 0.3659816029268629499455821 │
│┆ 6.49216506191430300 ┆ 0.3665061494881803974380080 │
│┆ 6.52547997712276100 ┆ 0.3670852494123281846890350 │
│┆ 6.52633668834955500 ┆ 0.3640433382025421930433322 │
│┆ 6.54062696173521900 ┆ 0.3625406868862515419391480 │
│┆ 6.56139003303141100 ┆ 0.3617801610551591658290261 │
│┆ 6.56146720127919000 ┆ 0.3587737652788547204731028 │
│┆ 6.57198311076433400 ┆ 0.3569500992410880361767082 │
│┆ 6.58027798136652800 ┆ 0.3549185107545667919407728 │
│┆ 6.64401939586809600 ┆ 0.3588861279078980434731498 │
│┆ 6.64420650330255400 ┆ 0.3560119359558705767557166 │
│┆ 6.64452897012813600 ┆ 0.3531981218789765410257301 │
│┆ 6.64473085523170600 ┆ 0.3504162550672085638097694 │
│┆ 6.64483590758214500 ┆ 0.3476680648715986507677159 │
│┆ 6.69896867751782900 ┆ 0.3505951667372263346199423 │
│┆ 6.72936823461178400 ┆ 0.3510418359457528557187069 │
│┆ 6.72939209982941400 ┆ 0.3483439848711271526037531 │
│┆ 6.72939824425483300 ┆ 0.3456855017540460230648202 │
│┆ 6.72940128194286000 ┆ 0.3430669819198046026173962 │
│┆ 6.85176106702289600 ┆ 0.3529821783426371002389518 │
│┆ 6.98829741881966500 ┆ 0.3644500060737432263501626 │
│┆ 7.01468968027914500 ┆ 0.3644879356341832111224083 │
│┆ 7.02024736009352800 ┆ 0.3623814190948540366925067 │
│┆ 7.02278514481558600 ┆ 0.3599964320455654833983500 │
│┆ 7.02592270446957400 ┆ 0.3577071728201525570422949 │
│┆ 7.02736287467670800 ┆ 0.3552793451250682390498958 │
│┆ 7.02738316576834000 ┆ 0.3527436725608875456255896 │
│┆ 7.02740761003779800 ┆ 0.3502443809760081986218935 │
│┆ 7.02764030817734200 ┆ 0.3478009035290065242185501 │
│┆ 7.02766234675907400 ┆ 0.3453708955248619248379818 │
│┆ 7.02834251626109800 ┆ 0.3430388786519679347538424 │
│┆ 7.11241134878905000 ┆ 0.3488716909957467134682855 │
│┆ 7.11355237724572300 ┆ 0.3465933385193032670584077 │
│┆ 7.11614248667694000 ┆ 0.3444862849706711781319995 │
│┆ 7.11617420772832800 ┆ 0.3421617253698506534135287 │
│┆ 7.11631631307915000 ┆ 0.3398789118644055492708333 │
│┆ 7.11695462793427900 ┆ 0.3376736211738343442252716 │
│┆ 7.16118212425472500 ┆ 0.3396194001108967921134879 │
│┆ 7.17611686605711600 ┆ 0.3387937715480881851173695 │
│┆ 7.17611686605711600 ┆ 0.3365794331719568897898050 │
│┆ 7.17611686605711600 ┆ 0.3343938524370740528431180 │
│┆ 7.17611737110805000 ┆ 0.3322365195091529716444179 │
│┆ 7.17611737110805000 ┆ 0.3301067982302481449031075 │
│┆ 7.39139446955125000 ┆ 0.3479790586274700914864738 │
│┆ 7.57426475393155700 ┆ 0.3630980162193029728537256 │
│┆ 7.57440705437024900 ┆ 0.3608279385238603282440879 │
│┆ 7.57744356175544700 ┆ 0.3588603183224317170826514 │
│┆ 7.57780981960385600 ┆ 0.3566658488328237553632843 │
│┆ 7.57805687549749700 ┆ 0.3544873210387338688119213 │
│┆ 7.65685950024974600 ┆ 0.3596778981997839881315115 │
│┆ 7.70096892792561300 ┆ 0.3616153806638766172726296 │
│┆ 7.71829079548821600 ┆ 0.3610425018407158675320108 │
│┆ 7.71831837172513000 ┆ 0.3588701113693353135075438 │
│┆ 7.71831837172513000 ┆ 0.3567211885467644433667801 │
│┆ 7.71831837172513000 ┆ 0.3545978481387479883467397 │
│┆ 7.72284980414667200 ┆ 0.3529136632982739057332905 │
│┆ 7.73171528602274400 ┆ 0.3516436544948103650642946 │
│┆ 7.73384110520468200 ┆ 0.3497795218745823225447316 │
│┆ 7.73944078186985600 ┆ 0.3482496721864551625683240 │
│┆ 7.74625792927120100 ┆ 0.3468468896346644788108613 │
│┆ 7.74830302120458900 ┆ 0.3450356305080928826124632 │
│┆ 7.75269759861920000 ┆ 0.3434532574607766303414092 │
│┆ 7.87847386648793200 ┆ 0.3526724458234846873693653 │
│┆ 7.88421881444026700 ┆ 0.3511915610958976798042967 │
│┆ 7.88422298951153100 ┆ 0.3492189446536075452855842 │
│┆ 7.88423883037294900 ┆ 0.3472693962813441724858013 │
│┆ 7.88425023035720100 ┆ 0.3453411205271532057385558 │
│┆ 7.88427035170653600 ┆ 0.3434349114850757175410357 │
│┆ 7.88429339560134100 ┆ 0.3415499030105655152431154 │
│┆ 8.00571094698414900 ┆ 0.3502262719489718019702648 │
│┆ 8.11154147240867200 ┆ 0.3575929622750317746579205 │
│┆ 8.11326743698327200 ┆ 0.3558113973189897918574487 │
│┆ 8.12045265209566000 ┆ 0.3545255444888571941149989 │
│┆ 8.12535134049676800 ┆ 0.3530552642059498645410964 │
│┆ 8.12645749072853200 ┆ 0.3512729327054141955145751 │
│┆ 8.16673760334064500 ┆ 0.3528867887926883718024546 │
│┆ 8.16903900871313900 ┆ 0.3512273596098786562250568 │
│┆ 8.17314345958562900 ┆ 0.3497396545076824311143685 │
│┆ 8.17314345958562900 ┆ 0.3479180938071215851189811 │
│┆ 8.17314345958562900 ┆ 0.3461154093832504888230279 │
│┆ 8.17314345958562900 ┆ 0.3443313093348832182620844 │
│┆ 8.17314492890279300 ┆ 0.3425656309171766217578607 │
│┆ 8.17314507027551300 ┆ 0.3408178588763720425276415 │
│┆ 8.26611037275322600 ┆ 0.3468455872819212024704949 │
│┆ 8.26611037275322600 ┆ 0.3450938418916084691246843 │
│┆ 8.26611037275322600 ┆ 0.3433597019826054114908919 │
│┆ 8.26611345139169800 ┆ 0.3416431579563938480710651 │'''
Paul_Jurczak_scores = [float(t.split(' ')[-1]) for t in Paul_Jurczak_scores.split('\n')]
Stanislav_Chistyakov_scores = [float(t.split(' ')[-2]) for t in Stanislav_Chistyakov_scores.split('\n')]

In [ ]:
# We can compare own scores with public ones and compute the best scores ensemble
# The main use is to suggest team but also we can point the most feasable improvements to our scores
plt.plot(Paul_Jurczak_scores)
plt.plot(Stanislav_Chistyakov_scores)
my_scores = [competition_score(None,df,None) for df in submission]
plt.plot(my_scores)
best_scores = [min(s1,s2,s3) for s1,s2,s3 in zip(Paul_Jurczak_scores,Stanislav_Chistyakov_scores,my_scores)]
ensemble_score = sum(best_scores)
ensemble_score

In [ ]:
# We can check were we know by sure there is more room for improvement
improvements = np.array([my_scores[k] - best_scores[k] for k in range(200)])
plt.plot(improvements)

In [ ]:
improvements[64] = 0 # We can skip problematic cases
k = np.argmax(improvements)
plot_results(
    [ChristmasTree(float(x[1:]),float(y[1:]),float(d[1:])) for x,y,d in submission[k][['x','y','deg']].values],
    k + 1
)
s = np.sqrt(best_scores[k] * (k + 1)) # Thats the larger size of public score solution
best_scores[k],s,improvements[k]

In [ ]:
# Gambling time using generated json as input
# https://jeroengar.github.io/sparroWASM/
import json

data = {
  "name": "ChristmasTree",
  "items": [
    {
      "id": 0,
      "demand": int(k+1),
      "dxf": "dxf/i_0.dxf",
      "shape": {
        "type": "simple_polygon",
        "data": [
            [0.0, 0.8],
            [0.125, 0.5],
            [0.0625, 0.5],
            [0.2, 0.25],
            [0.1, 0.25],
            [0.35, 0.0],
            [0.075, 0.0],
            [0.075, -0.2],
            [-0.075, -0.2],
            [-0.075, 0.0],
            [-0.35, 0.0],
            [-0.1, 0.25],
            [-0.2, 0.25],
            [-0.0625, 0.5],
            [-0.125, 0.5]
        ]
      }
    }
  ],
  "strip_height": float(s) # We fix h to the larger side observed in public score solution
}

with open('christmas_tree_config.json', 'w') as f:
    json.dump(data, f, indent=2)

In [ ]:
# Once found a best than your own solution upload it and edit the path
src_pth = f'/kaggle/input/sparrowasm-example/sparroWASM_{k+1}_solution.svg'

In [ ]:
import xml.etree.ElementTree as ET

def inspect_svg_structure(svg_file):
    tree = ET.parse(svg_file)
    root = tree.getroot()
    
    print("=== SVG ROOT ATTRIBUTES ===")
    for key, value in root.items():
        print(f"{key}: {value}")
    
    print("\n=== ALL ELEMENTS ===")
    for elem in root.iter():
        tag = elem.tag.split('}')[-1]  # Remove namespace
        print(f"\n<{tag}>")
        for attr, value in elem.items():
            print(f"  {attr}: {value}")
        if elem.text and elem.text.strip():
            print(f"  Text: {elem.text[:50]}...")
        
        # Show first few elements only
        if len(list(root.iter())) > 50:
            print("\n[... more elements ...]")
            break

# Run this
inspect_svg_structure(src_pth)

In [ ]:
import xml.etree.ElementTree as ET
import re

def debug_svg_structure(svg_file, max_elements=100):
    """Debug the actual SVG structure with more detail."""
    tree = ET.parse(svg_file)
    root = tree.getroot()
    
    # Remove namespaces for easier parsing
    for elem in root.iter():
        if '}' in elem.tag:
            elem.tag = elem.tag.split('}', 1)[1]
    
    print("=== DEEP SVG INSPECTION ===")
    print(f"Root: {root.tag}")
    
    count = 0
    for elem in root.iter():
        count += 1
        if count > max_elements:
            print(f"\n[... stopped at {max_elements} elements ...]")
            break
            
        tag = elem.tag
        attrs = dict(elem.items())
        
        # Only show elements with attributes or specific tags
        if attrs or tag in ['polygon', 'path', 'g', 'rect', 'circle']:
            print(f"\n[{count}] <{tag}>")
            for attr, value in attrs.items():
                if attr == 'points' and len(value) > 100:
                    print(f"  {attr}: {value[:100]}...")
                elif attr == 'd' and len(value) > 100:  # path data
                    print(f"  {attr}: {value[:100]}...")
                else:
                    print(f"  {attr}: {value}")
            
            # Check for nested elements
            children = list(elem)
            if children:
                print(f"  Has {len(children)} children")

# Run it
debug_svg_structure(src_pth, max_elements=50)

In [ ]:
import xml.etree.ElementTree as ET
import re
import json
import math

def extract_tree_positions_from_use_elements(svg_file):
    """
    Extract center_x, center_y, and rotation from <use> elements with transforms.
    """
    tree = ET.parse(svg_file)
    root = tree.getroot()
    
    # Remove namespaces for easier parsing
    for elem in root.iter():
        if '}' in elem.tag:
            elem.tag = elem.tag.split('}', 1)[1]
    
    tree_data = []
    
    # Find all <use> elements that reference item_0
    for elem in root.iter('use'):
        href = elem.get('href') or elem.get('xlink:href', '')
        if '#item_0' in href:
            transform = elem.get('transform', '')
            if transform:
                # Parse transform: "translate(x y), rotate(angle)"
                tx, ty, angle = parse_use_transform(transform)
                
                if tx is not None and ty is not None:
                    tree_data.append({
                        'center_x': tx,
                        'center_y': ty,
                        'angle': angle
                    })
    
    return tree_data

def parse_use_transform(transform_str):
    """
    Parse transform string like: "translate(1.7997283 0.21162309), rotate(-12.499795)"
    Returns: (center_x, center_y, angle)
    """
    tx, ty, angle = None, None, 0.0
    
    # Parse translation (note: space-separated, not comma-separated!)
    translate_match = re.search(r'translate\(([-\d\.]+)\s+([-\d\.]+)\)', transform_str)
    if translate_match:
        tx = float(translate_match.group(1))
        ty = float(translate_match.group(2))
    
    # Parse rotation
    rotate_match = re.search(r'rotate\(([-\d\.]+)\)', transform_str)
    if rotate_match:
        angle = float(rotate_match.group(1))
    
    return tx, ty, angle

def scale_to_original_coordinates(tree_data, original_scale=800000000000000.0):
    """
    Scale coordinates back to your original coordinate system.
    Your SVG coordinates are scaled down, original tree was 0.8 units tall.
    """
    # Your original tree height was 0.8 in SVG, but 800000000000000 in original
    svg_height = 0.8  # In your SVG, tree is 0.8 units tall
    original_height = original_scale  # Your original coordinate system
    
    scale_factor = original_height / svg_height
    
    scaled_data = []
    for tree in tree_data:
        scaled_data.append({
            'center_x': tree['center_x'] * scale_factor,
            'center_y': tree['center_y'] * scale_factor,
            'angle': tree['angle']
        })
    
    return scaled_data

# MAIN EXTRACTION
if __name__ == "__main__":
    svg_file = src_pth  # Your SVG file
    
    # Extract tree positions
    trees = extract_tree_positions_from_use_elements(svg_file)
    
    print(f"Found {len(trees)} Christmas trees")
    
    # Scale back to original coordinates (if needed)
    # trees_scaled = scale_to_original_coordinates(trees)
    # trees = trees_scaled  # Uncomment if you need original scale
    
    # Save in your desired format
    output = {
        "name": "OptimizedChristmasTrees",
        "items": [
            {
                "id": i,
                "center_x": str(tree['center_x']),  # Keep as string for Decimal
                "center_y": str(tree['center_y']),
                "angle": str(tree['angle'])
            }
            for i, tree in enumerate(trees)
        ]
    }
    
    with open("extracted_tree_positions.json", "w") as f:
        json.dump(output, f, indent=2)
    
    print(f"\n=== FIRST 5 TREES ===")
    for i, tree in enumerate(trees[:5]):
        print(f"Tree {i}: center=({tree['center_x']:.6f}, {tree['center_y']:.6f}), angle={tree['angle']:.2f}°")
    
    print(f"\nSaved to 'extracted_tree_positions.json'")

In [ ]:
df = submission[len(trees)-1].copy()
print(competition_score(None,df,None))
df['x'] = [f's{t["center_x"]}' for t in trees]
df['y'] = [f's{t["center_y"]}' for t in trees]
df['deg'] = [f's{t["angle"]}' for t in trees]
print(competition_score(None,df,None))
plot_results(
    [ChristmasTree(float(x[1:]),float(y[1:]),float(d[1:])) for x,y,d in df[['x','y','deg']].values],
    len(df)
)
submission[len(trees)-1] = df
df = pd.concat(submission)
print(competition_score(None,df,None))
df.to_csv('submission.csv',index=False)